In [1]:
import nfl_data_py as nfl
import pandas as pd
import numpy as np

In [ ]:
pbp = nfl.import_pbp_data([2024])
weekly_roster_data = nfl.import_weekly_rosters([2024])
roster_data = nfl.import_rosters([2024])
earnings_data = pd.read_csv("overthecap/overthecap-earnings.csv")

2024 done.
Downcasting floats.


In [31]:
wr_data = weekly_roster_data[['team', 'week', 'player_name', 'player_id', 'position', 'status']]

wr_data_with_earnings = pd.merge(
    wr_data,
    earnings_data,
    left_on='player_name',
    right_on='player_name',
    how='left'
)

# Fix: use .str.contains instead of .contains, and handle possible NaNs
wr_earnings_data = wr_data_with_earnings[["player_name", "position", "team_x", "week", "status", "capnumber"]]

wr_earnings_data[wr_earnings_data["status"] == "ACT"]



,player_name,position,team_x,week,status,capnumber
13,Aaron Rodgers,QB,NYJ,9,ACT,17161112.0
14,Aaron Rodgers,QB,NYJ,8,ACT,17161112.0
15,Aaron Rodgers,QB,NYJ,7,ACT,17161112.0
16,Aaron Rodgers,QB,NYJ,4,ACT,17161112.0
17,Aaron Rodgers,QB,NYJ,13,ACT,17161112.0
...,...,...,...,...,...,...
46659,Jalen Sundell,OL,SEA,14,ACT,795000.0
46660,Jalen Sundell,OL,SEA,13,ACT,795000.0
46729,Jude McAtamney,K,NYG,9,ACT,31667.0
46822,Jamie Sheriff,LB,CAR,1,ACT,112500.0


In [ ]:

find_injuries = pbp[pbp['desc'].str.lower().str.contains("injur")]

for index, play in find_injuries.iterrows():
    desc_sentences = play['desc'].split('. ')
    print(desc_sentences.filter(lambda x: "injur" in x.lower()))

['BUF-7-Ta.Johnson was injured during the play']
['ARI-73-J.Williams was injured during the play']
['** Injury Update: ARI-74-I.Adams has returned to the game.']
['BAL-53-K.Van Noy was injured during the play']
['KC-54-L.Chenal was injured during the play.']
['BAL-80-I.Likely was injured during the play.']
['** Injury Update: KC-54-L.Chenal has returned to the game.']
['** Injury Update: BAL-80-I.Likely has returned to the game.']
['NO-87-F.Moreau was injured during the play.']
['CLE-85-D.Njoku was injured during the play']
['DAL-87-J.Ferguson was injured during the play.']
['CLE-42-T.Fields was injured during the play.']
['** Injury Update: CLE-42-T.Fields has returned to the game', '** Injury Update: DAL-24-I.Mukuamu has returned to the game.']
['** Injury Update: DAL-50-E.Kendricks has returned to the game.']
['CLE-23-M.Emerson was injured during the play.']
['SEA-74-G.Fant was injured during the play']
['PHI-8-C.Gardner-Johnson was injured during the play', 'PHI-27-Q.Mitchell was i